# Лабораторная работа №6

## Задание

Провести классификацию найденного датасета, методами CatBoost

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler

Загрузка датасета

In [2]:
df = pd.read_csv('cars.csv', encoding='utf-8')
print(df.head(5))

  manufacturer_name model_name transmission   color  odometer_value  \
0            Subaru    Outback    automatic  silver          190000   
1            Subaru    Outback    automatic    blue          290000   
2            Subaru   Forester    automatic     red          402000   
3            Subaru    Impreza   mechanical    blue           10000   
4            Subaru     Legacy    automatic   black          280000   

   year_produced engine_fuel  engine_has_gas engine_type  engine_capacity  \
0           2010    gasoline           False    gasoline              2.5   
1           2002    gasoline           False    gasoline              3.0   
2           2001    gasoline           False    gasoline              2.5   
3           1999    gasoline           False    gasoline              3.0   
4           2001    gasoline           False    gasoline              2.5   

   ... feature_1  feature_2 feature_3 feature_4  feature_5  feature_6  \
0  ...      True       True      True

Удаление ненужных столбцов из датасета

In [3]:
columns_to_drop = ['feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'duration_listed']
df = df.drop(columns=columns_to_drop, axis=1)
print(df.head(5))

  manufacturer_name model_name transmission   color  odometer_value  \
0            Subaru    Outback    automatic  silver          190000   
1            Subaru    Outback    automatic    blue          290000   
2            Subaru   Forester    automatic     red          402000   
3            Subaru    Impreza   mechanical    blue           10000   
4            Subaru     Legacy    automatic   black          280000   

   year_produced engine_fuel  engine_has_gas engine_type  engine_capacity  \
0           2010    gasoline           False    gasoline              2.5   
1           2002    gasoline           False    gasoline              3.0   
2           2001    gasoline           False    gasoline              2.5   
3           1999    gasoline           False    gasoline              3.0   
4           2001    gasoline           False    gasoline              2.5   

   body_type  has_warranty  state drivetrain  price_usd  is_exchangeable  \
0  universal         False  owned 

Удаление строк с пропущенными значениями

In [4]:
df.dropna(inplace=True)

Предобработка данных

In [5]:
# Кодирование категориальных признаков
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])
    
print(df.head(5))

   manufacturer_name  model_name  transmission  color  odometer_value  \
0                 45         763             0      8          190000   
1                 45         763             0      1          290000   
2                 45         519             0      7          402000   
3                 45         609             1      1           10000   
4                 45         664             0      0          280000   

   year_produced  engine_fuel  engine_has_gas  engine_type  engine_capacity  \
0           2010            2           False            1              2.5   
1           2002            2           False            1              3.0   
2           2001            2           False            1              2.5   
3           1999            2           False            1              3.0   
4           2001            2           False            1              2.5   

   body_type  has_warranty  state  drivetrain  price_usd  is_exchangeable  \
0        

In [6]:
# Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = df.select_dtypes(include=['int32', 'int64', 'float32', 'float64']).columns
df[numeric_features] = scaler.fit_transform(df[numeric_features])

print(df.head(5))

   manufacturer_name  model_name  transmission     color  odometer_value  \
0           1.089297    0.597314     -1.410285  0.978100       -0.432979   
1           1.089297    0.597314     -1.410285 -0.969685        0.302004   
2           1.089297   -0.149960     -1.410285  0.699845        1.125184   
3           1.089297    0.125674      0.709076 -0.969685       -1.755946   
4           1.089297    0.294117     -1.410285 -1.247940        0.228505   

   year_produced  engine_fuel  engine_has_gas  engine_type  engine_capacity  \
0       0.875318     0.721021           False     0.708498         0.662782   
1      -0.116665     0.721021           False     0.708498         1.407751   
2      -0.240663     0.721021           False     0.708498         0.662782   
3      -0.488659     0.721021           False     0.708498         1.407751   
4      -0.240663     0.721021           False     0.708498         0.662782   

   body_type  has_warranty     state  drivetrain  price_usd  is_exch

Разделение данных на обучающий и тестовый наборы

In [7]:
X = df.drop('feature_0', axis=1)
Y = df['feature_0']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

##### Обучение модели методами CatBoost

Создание объекта CatBoostClassifier

In [8]:
model = CatBoostClassifier()

Определение сетки параметров для поиска

In [9]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'n_estimators': [50, 100, 200]
}

Поиск оптимальных параметров с использованием кросс-валидации

In [18]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, Y_train, verbose=False)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001B256168B90>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'learning_rate': [0.01, 0.05, 0.1],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

Получение лучших параметров

In [25]:
best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)

Лучшие параметры: {'depth': 8, 'learning_rate': 0.1, 'n_estimators': 200}


Обучение модели с лучшими параметрами

In [20]:
best_classifier = CatBoostClassifier(**best_params)
best_classifier.fit(X_train, Y_train, verbose=False)

Предсказание на тестовом наборе данных

In [22]:
Y_pred = best_classifier.predict(X_test)

Оценка модели

In [23]:
print("Отчет о классификации:")
print(classification_report(Y_test, Y_pred))

Отчет о классификации:
              precision    recall  f1-score   support

       False       0.88      0.94      0.91      5958
        True       0.74      0.54      0.63      1747

    accuracy                           0.85      7705
   macro avg       0.81      0.74      0.77      7705
weighted avg       0.85      0.85      0.84      7705



Оценка модели на тестовом наборе данных

In [26]:
accuracy = accuracy_score(Y_test, Y_pred)
print("Точность модели методами CatBoost:")
print(accuracy)

Точность модели методами CatBoost:
0.8533419857235561
